# Atividade: Leitura, Limpeza/Normalização e Exportação para Excel

**Objetivo**: Ler um arquivo `.csv` ou `.txt`, realizar sanitização e normalização (e-mail, telefone, CEP, datas, valores, booleanos), e exportar para `resultado_limpeza.xlsx` com abas `dados_limpos`, `erros` e `dicionario_dados`.

**Arquivos fornecidos**: `dados_clientes_sujos.csv` e `dados_clientes_sujos.txt`.


In [ ]:
import pandas as pd, numpy as np, re, unicodedata
from datetime import datetime
ENTRADA_ARQUIVO = 'dados_clientes_sujos.csv'  # ou 'dados_clientes_sujos.txt'
SEPARADOR_TXT = ';'
SAIDA_XLSX = 'resultado_limpeza.xlsx'


In [ ]:
if ENTRADA_ARQUIVO.endswith('.txt'):
    df = pd.read_csv(ENTRADA_ARQUIVO, sep=SEPARADOR_TXT, encoding='utf-8')
else:
    df = pd.read_csv(ENTRADA_ARQUIVO, encoding='utf-8')
df.head()

In [ ]:
def strip_and_normalize(s):
    if pd.isna(s):
        return s
    s = str(s)
    s = ''.join(ch for ch in s if ch.isprintable())
    s = s.strip()
    s = unicodedata.normalize('NFC', s)
    s = re.sub(r'\s{2,}', ' ', s)
    return s
for col in df.columns:
    df[col] = df[col].apply(strip_and_normalize)
df.head()

In [ ]:
# Exemplos: UF, e-mail, telefone, CEP, datas, booleanos, dinheiro
uf_validas = set(['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SE','SP','TO'])
if 'estado' in df.columns:
    df['estado'] = df['estado'].str.upper().str[:2]
    df.loc[~df['estado'].isin(uf_validas), 'estado'] = np.nan

email_re = re.compile(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$')
if 'email' in df.columns:
    df['email'] = df['email'].str.lower()
    df.loc[~df['email'].fillna('').str.match(email_re), 'email'] = np.nan

def limpa_telefone(s):
    if pd.isna(s) or s == '':
        return np.nan, 'invazio'
    dig = re.sub(r'\D', '', str(s))
    if dig.startswith('55') and len(dig) in (12,13):
        dig = dig[2:]
    if len(dig) == 10:
        return f'({dig[:2]}) {dig[2:6]}-{dig[6:]}', None
    if len(dig) == 11:
        return f'({dig[:2]}) {dig[2:7]}-{dig[7:]}', None
    return np.nan, 'invalido'
if 'telefone' in df.columns:
    t = df['telefone'].apply(limpa_telefone)
    df['telefone'] = t.apply(lambda x: x[0])
    df['_telefone_erro'] = t.apply(lambda x: x[1])

def padroniza_cep(s):
    if pd.isna(s) or s == '':
        return np.nan, 'invazio'
    dig = re.sub(r'\D','', str(s))
    if len(dig)==8:
        return f'{dig[:5]}-{dig[5:]}', None
    return np.nan, 'invalido'
if 'cep' in df.columns:
    c = df['cep'].apply(padroniza_cep)
    df['cep'] = c.apply(lambda x: x[0])
    df['_cep_erro'] = c.apply(lambda x: x[1])

def parse_data(s):
    if pd.isna(s) or s == '':
        return np.nan
    s = s.strip()
    for fmt in ('%d/%m/%Y','%Y-%m-%d','%d-%m-%Y','%m/%d/%Y'):
        try:
            return pd.to_datetime(s, format=fmt, dayfirst=True, errors='raise')
        except Exception:
            pass
    return pd.to_datetime(s, errors='coerce', dayfirst=True)
for col in ('data_nascimento','data_compra_mais_recente'):
    if col in df.columns:
        df[col] = df[col].apply(parse_data)

map_bool = {'sim':True,'yes':True,'true':True,'1':True,
           'nao':False,'não':False,'no':False,'false':False,'0':False}
def to_bool(s):
    if pd.isna(s): return np.nan
    k = str(s).strip().lower().replace('ã','a')
    return map_bool.get(k, np.nan)
if 'ativo' in df.columns:
    df['ativo'] = df['ativo'].apply(to_bool)

def parse_money(s):
    if pd.isna(s) or s=='': return np.nan
    s = str(s).strip().replace('R$','').replace(' ','')
    s = s.replace('.','').replace(',', '.')
    s = re.sub(r'[^0-9\.-]','', s)
    try:
        v = float(s)
        return np.nan if v<0 else v
    except:
        return np.nan
for col in ('renda_mensal','valor_compra_mais_recente'):
    if col in df.columns:
        df[col] = df[col].apply(parse_money)
df.head()

In [ ]:
erros = pd.DataFrame(index=df.index)
for c in ('id','nome','email','telefone','estado','cep'):
    if c in df.columns:
        erros[f'{c}_nulo'] = df[c].isna()
if 'data_nascimento' in df.columns:
    idade = (pd.Timestamp('today').normalize() - df['data_nascimento']).dt.days / 365.25
    df['idade_estimada'] = idade
    erros['idade_fora_faixa'] = (idade < 15) | (idade > 100)
def flag_outlier(s):
    q1,q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3-q1
    return (s < (q1-1.5*iqr)) | (s > (q3+1.5*iqr))
for col in ('renda_mensal','valor_compra_mais_recente'):
    if col in df.columns:
        erros[f'{col}_outlier'] = flag_outlier(df[col])
if 'email' in df.columns:
    erros['email_duplicado'] = df['email'].duplicated(keep='first')
df_limpo = df.copy()
if 'email' in df_limpo.columns:
    df_limpo = df_limpo[~df_limpo['email'].duplicated(keep='first')]
dicionario = pd.DataFrame({
    'coluna': df_limpo.columns,
    'descricao': [
        'Identificador sequencial', 'Nome', 'E-mail', 'Telefone formatado', 'Data de nascimento', 'Cidade', 'UF', 'CEP',
        'Renda mensal (float)', 'Data compra mais recente', 'Valor compra mais recente (float)', 'Ativo (bool)', 'Categoria', 'Idade estimada'
    ][:len(df_limpo.columns)]
})
with pd.ExcelWriter(SAIDA_XLSX, engine='openpyxl') as xl:
    df_limpo.to_excel(xl, sheet_name='dados_limpos', index=False)
    erros.reset_index(names='idx').to_excel(xl, sheet_name='erros', index=False)
    dicionario.to_excel(xl, sheet_name='dicionario_dados', index=False)
SAIDA_XLSX